In [10]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier, StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pickle

In [11]:
N_SAMPLES = 1000

In [12]:
df = pd.read_csv('./datasets/csgo_processed.csv', sep=';')

orig_X = df.drop(['bomb_planted'], axis=1)[:N_SAMPLES].values
orig_y = df['bomb_planted'][:N_SAMPLES].values

oversampler = SMOTE()
X, y = oversampler.fit_resample(orig_X, orig_y)

ss = StandardScaler()
ss.fit(X)
with open("./models/scaler.pkl", "wb") as f:
    pickle.dump(ss, f)
X = ss.fit_transform(X)
categorical_y = tf.keras.utils.to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
cat_X_train, cat_X_test, cat_y_train, cat_y_test = train_test_split(X, categorical_y, test_size=0.2, random_state=42, stratify=categorical_y)

In [13]:
gnb = GaussianNB()
cbc = CatBoostClassifier(learning_rate=0.2, n_estimators=100, max_depth=5)
bc = BaggingClassifier(n_estimators=100, max_features=0.5, max_samples=0.75)
sc = StackingClassifier(cv=5, final_estimator=LogisticRegression(), estimators=[('lr', LogisticRegression()), ('rf', RandomForestClassifier()), ('svc', SVC(probability=True))])
xgbc = XGBClassifier()
nn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [14]:
gnb.fit(X_train, y_train)
print("GNB score:", gnb.score(X_test, y_test))
cbc.fit(X_train, y_train)
print("GBC score:", cbc.score(X_test, y_test))
bc.fit(X_train, y_train)
print("BC score:", bc.score(X_test, y_test))
sc.fit(X_train, y_train)
print("SC score:", sc.score(X_test, y_test))
xgbc.fit(X_train, y_train)
print("XGBC score:", xgbc.score(X_test, y_test))

GNB score: 0.9217391304347826
0:	learn: 0.5545761	total: 157ms	remaining: 15.6s
1:	learn: 0.4520985	total: 160ms	remaining: 7.82s
2:	learn: 0.3817653	total: 161ms	remaining: 5.21s
3:	learn: 0.3290506	total: 164ms	remaining: 3.95s
4:	learn: 0.2841084	total: 167ms	remaining: 3.16s
5:	learn: 0.2501411	total: 168ms	remaining: 2.64s
6:	learn: 0.2162053	total: 170ms	remaining: 2.26s
7:	learn: 0.1942064	total: 172ms	remaining: 1.98s
8:	learn: 0.1769506	total: 174ms	remaining: 1.75s
9:	learn: 0.1626193	total: 175ms	remaining: 1.58s
10:	learn: 0.1508669	total: 177ms	remaining: 1.43s
11:	learn: 0.1404037	total: 178ms	remaining: 1.31s
12:	learn: 0.1302205	total: 180ms	remaining: 1.2s
13:	learn: 0.1215195	total: 181ms	remaining: 1.11s
14:	learn: 0.1144003	total: 182ms	remaining: 1.03s
15:	learn: 0.1094207	total: 183ms	remaining: 962ms
16:	learn: 0.1027803	total: 184ms	remaining: 900ms
17:	learn: 0.0970909	total: 186ms	remaining: 846ms
18:	learn: 0.0938788	total: 187ms	remaining: 796ms
19:	learn: 0

In [15]:
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn.fit(cat_X_train, cat_y_train, epochs=100, batch_size=32, validation_data=(cat_X_test, cat_y_test))
loss, accuracy = nn.evaluate(cat_X_test, cat_y_test, verbose=0)
print("NN test accuracy:", accuracy)

Epoch 1/100
44/44 [==============================] - 1s 6ms/step - loss: 0.4113 - accuracy: 0.8293 - val_loss: 0.3018 - val_accuracy: 0.8725
Epoch 2/100
44/44 [==============================] - 0s 2ms/step - loss: 0.2219 - accuracy: 0.9150 - val_loss: 0.2563 - val_accuracy: 0.8841
Epoch 3/100
44/44 [==============================] - 0s 2ms/step - loss: 0.1968 - accuracy: 0.9230 - val_loss: 0.2386 - val_accuracy: 0.8957
Epoch 4/100
44/44 [==============================] - 0s 2ms/step - loss: 0.1751 - accuracy: 0.9325 - val_loss: 0.2039 - val_accuracy: 0.9246
Epoch 5/100
44/44 [==============================] - 0s 2ms/step - loss: 0.1549 - accuracy: 0.9426 - val_loss: 0.1767 - val_accuracy: 0.9362
Epoch 6/100
44/44 [==============================] - 0s 3ms/step - loss: 0.1387 - accuracy: 0.9463 - val_loss: 0.1657 - val_accuracy: 0.9478
Epoch 7/100
44/44 [==============================] - 0s 2ms/step - loss: 0.1219 - accuracy: 0.9579 - val_loss: 0.1403 - val_accuracy: 0.9594
Epoch 8/100
4

In [16]:
with open('./models/gnb.pkl', 'wb') as f:
    pickle.dump(gnb, f)
with open('./models/cbc.pkl', 'wb') as f:
    pickle.dump(cbc, f)
with open('./models/bc.pkl', 'wb') as f:
    pickle.dump(bc, f)
with open('./models/sc.pkl', 'wb') as f:
    pickle.dump(sc, f)
with open('./models/xgbc.pkl', 'wb') as f:
    pickle.dump(xgbc, f)
nn.save('./models/nn_model.h5')

c:\Users\csr10\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
with open('./models/gnb.pkl', 'rb') as f:
    gnb = pickle.load(f)
with open('./models/cbc.pkl', 'rb') as f:
    cbc = pickle.load(f)
with open('./models/bc.pkl', 'rb') as f:
    bc = pickle.load(f)
with open('./models/sc.pkl', 'rb') as f:
    sc = pickle.load(f)
with open('./models/xgbc.pkl', 'rb') as f:
    xgbc = pickle.load(f)
nn = tf.keras.models.load_model('./models/nn_model.h5')